In [2]:
#!pip install dash==2.0.0

Exception in thread Thread-4:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\dash\dash.py", line 1747, in <lambda>
    if debug and dev_tools.prune_errors:
  File "C:\ProgramData\Anaconda3\lib\site-packages\dash\_watch.py", line 35, in watch
    walk()
  File "C:\ProgramData\Anaconda3\lib\site-packages\dash\_watch.py", line 20, in walk
    info = os.stat(path)
FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\dash\\html\\__pycache__\\Textarea.cpython-38.pyc.1327098396512'


  Attempting uninstall: dash
    Found existing installation: dash 2.2.0
    Uninstalling dash-2.2.0:
      Successfully uninstalled dash-2.2.0


In [5]:
#!pip install dash-leaflet==0.1.22
!pip install dash-leaflet==0.1.23

  Using cached dash_leaflet-0.1.23-py3-none-any.whl (1.9 MB)
  Attempting uninstall: dash-leaflet
    Found existing installation: dash-leaflet 0.1.22
    Uninstalling dash-leaflet-0.1.22:
      Successfully uninstalled dash-leaflet-0.1.22


In [1]:
#!pip install dash-leaflet==0.1.23


In [3]:
import dash
dash.__version__

'2.0.0'

In [1]:
import dash_leaflet as dl
import dash_leaflet.express as dlx
import pandas as pd
from dash_extensions.javascript import assign
from dash import Dash, html

colorscale = ['red', 'yellow', 'green', 'blue', 'purple']  # rainbow
chroma = "https://cdnjs.cloudflare.com/ajax/libs/chroma-js/2.1.0/chroma.min.js"  # js lib used for colors
color_prop = 'density'
# Pre process the data into geobuf.
df = pd.read_csv("assets/uscities.csv")  # data from https://simplemaps.com/data/us-cities
df = df[df['state_name'] == 'Washington']  # select one state
df = df[['lat', 'lng', 'city', color_prop]]  # drop irrelevant columns
dicts = df.to_dict('rows')
for item in dicts:
    item["tooltip"] = "{} ({:.1f})".format(item['city'], item[color_prop])  # bind tooltip
geojson = dlx.dicts_to_geojson(dicts, lon="lng")  # convert to geojson
geobuf = dlx.geojson_to_geobuf(geojson)  # convert to geobuf
# Create a colorbar.
vmax = df[color_prop].max()
colorbar = dl.Colorbar(colorscale=colorscale, width=20, height=150, min=0, max=vmax, unit='/km2')
# Geojson rendering logic, must be JavaScript as it is executed in clientside.
point_to_layer = assign("""function(feature, latlng, context){
    const {min, max, colorscale, circleOptions, colorProp} = context.props.hideout;
    const csc = chroma.scale(colorscale).domain([min, max]);  // chroma lib to construct colorscale
    circleOptions.fillColor = csc(feature.properties[colorProp]);  // set color based on color prop.
    return L.circleMarker(latlng, circleOptions);  // sender a simple circle marker.
}""")
# Create geojson.
geojson = dl.GeoJSON(data=geobuf, id="geojson", format="geobuf",
                     zoomToBounds=True,  # when true, zooms to bounds when data changes
                     options=dict(pointToLayer=point_to_layer),  # how to draw points
                     superClusterOptions=dict(radius=50),   # adjust cluster size
                     hideout=dict(colorProp=color_prop, circleOptions=dict(fillOpacity=1, stroke=False, radius=5),
                                  min=0, max=vmax, colorscale=colorscale))
# Create the app.
app = Dash(external_scripts=[chroma], prevent_initial_callbacks=True)
app.layout = html.Div([
    dl.Map([dl.TileLayer(), geojson, colorbar]),
], style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block", "position": "relative"})

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


C:\Users\Andras\anaconda3\envs\interdisciplinary_leaflet\lib\site-packages\pandas\core\frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Mar/2022 19:11:17] "GET /_reload-hash HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2022 19:11:17] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2022 19:11:18] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2022 19:11:20] "GET /_reload-hash HTTP/1.1" 200 -


In [5]:
import dash_leaflet as dl
import dash_leaflet.express as dlx
from dash import Dash, html, dcc, Output, Input
from dash_extensions.javascript import assign

# A few cities in Denmark.
cities = [dict(name="Aalborg", lat=57.0268172, lon=9.837735),
          dict(name="Aarhus", lat=56.1780842, lon=10.1119354),
          dict(name="Copenhagen", lat=55.6712474, lon=12.5237848)]
# Create drop down options.
dd_options = [dict(value=c["name"], label=c["name"]) for c in cities]
#dd_defaults = [o["value"] for o in dd_options]
lst = [o["value"] for o in dd_options]

lst_vals = ','.join([str(s) for s in lst])
dd_defaults = dict()
dd_defaults['values'] = [o["value"] for o in dd_options]
#print(dd_defaults)
# Generate geojson with a marker for each city and name as tooltip.
geojson = dlx.dicts_to_geojson([{**c, **dict(tooltip=c['name'])} for c in cities])
# Create javascript function that filters on feature name.
#geojson_filter = assign("""function(feature, context){
#    var nameArr = context.props.hideout.split(',');
#    return nameArr.includes(feature.properties.name);}""")
geojson_filter = assign("""function(feature, context){
    console.log(context.props.hideout)
    console.log(feature.properties.name)
    var nameArr = context.props.hideout.split(',');
    return feature.properties.name.filter(item => !nameArr.includes(item))
    ;}""")

dd_defaults
# Create example app.
app = Dash()
app.layout = html.Div([
    dl.Map(children=[
        dl.TileLayer(),
        dl.GeoJSON(data=geojson, hideout="Aalborg,Aarhus,Copenhagen",options=dict(filter=dd_defaults), id="geojson", zoomToBounds=True)
    ], style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}, id="map"),
    dcc.Dropdown(id="dd", value=lst, options=dd_options, clearable=False, multi=True)
])
# Link drop down to geojson hideout prop (could be done with a normal callback, but clientside is more performant).
#app.clientside_callback("function(x){return x;}", Output("geojson", "hideout"), Input("dd", "value"))
@app.callback(
    #Output(component_id='tbl', component_property='data'), #Output will be the table (later map)
    Output("geojson", "hideout"),
    Input("dd", "value")
)
def apply_filter(val):
    v_list = ','.join([str(s) for s in val])

    return v_list

    

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)
    #app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [3]:
import dash_leaflet as dl
import dash_leaflet.express as dlx
from dash import Dash, html, dcc, Output, Input
from dash_extensions.javascript import assign

# A few cities in Denmark.
cities = [dict(name="Aalborg", lat=57.0268172, lon=9.837735),
          dict(name="Aarhus", lat=56.1780842, lon=10.1119354),
          dict(name="Copenhagen", lat=55.6712474, lon=12.5237848)]
# Create drop down options.
dd_options = [dict(value=c["name"], label=c["name"]) for c in cities]
dd_defaults = [o["value"] for o in dd_options]
# Generate geojson with a marker for each city and name as tooltip.
geojson = dlx.dicts_to_geojson([{**c, **dict(tooltip=c['name'])} for c in cities])
# Create javascript function that filters on feature name.
geojson_filter = assign("function(feature, context){return context.props.hideout.includes(feature.properties.name);}")
# Create example app.
app = Dash()
app.layout = html.Div([
    dl.Map(children=[
        dl.TileLayer(),
        dl.GeoJSON(data=geojson, options=dict(filter=geojson_filter), hideout=dd_defaults, id="geojson", zoomToBounds=True)
    ], style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}, id="map"),
    dcc.Dropdown(id="dd", value=dd_defaults, options=dd_options, clearable=False, multi=True)
])
# Link drop down to geojson hideout prop (could be done with a normal callback, but clientside is more performant).
app.clientside_callback("function(x){return x;}", Output("geojson", "hideout"), Input("dd", "value"))

if __name__ == '__main__':
    #app.run_server(debug=True, use_reloader=False)
    #app.run_server(debug=True, use_reloader=False)
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [10]:
dd_defaults['values'] 

['Aalborg', 'Aarhus', 'Copenhagen']

In [13]:
lst_vals

'Aalborg,Aarhus,Copenhagen'

In [1]:
import base64
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px

import dash_bootstrap_components as dbc
#from dash import  dash_table
import dash_table
from datetime import date
from dash.dependencies import Input,Output,State
from dash import callback_context
from geopy.geocoders import Nominatim
import dash_leaflet as dl
from dash_extensions.javascript import assign
geolocator = Nominatim(user_agent="geoapiExercises")

info_colnames = ['trace_name','location','latitude','longitude','event_recorded_at','earthquake_depth','earthquake_magnitude']
#info_colnames = ['trace_name','latitude','longitude','event_recorded_at','earthquake_depth','earthquake_magnitude']
def create_event_infos(x):
    #'time','source_depth_km','source_magnitude','trace_name','source_latitude','source_longitude','trace_category'
    #selected_record = df_test.loc[x['trace_name']]
    selected_record = df_test.loc[x]
    latitude = str(selected_record['source_latitude'])
    longitude = str(selected_record['source_longitude'])
    address_details = ['road','county','state', 'country']
    try:
        location = geolocator.reverse(latitude + "," + longitude).raw
        location_string = str()
        for a in address_details:
            try:
                location_string += location['address'][a]
                location_string += ', '

            except KeyError:
                pass
        
        location_string = location_string[:-2]
    except AttributeError:
        location_string = "N/A"


    #print(location)
    info_dict = dict()
    info_dict['trace_name'] = x
    info_dict['location'] = location_string
    info_dict['latitude'] = latitude
    info_dict['longitude'] = longitude
    info_dict['event_recorded_at'] = str(selected_record['time'])
    info_dict['earthquake_depth'] = f'{selected_record["source_depth_km"]} km'
    info_dict['earthquake_magnitude'] = f'{selected_record["source_magnitude"]} km'

    return pd.DataFrame.from_dict(info_dict, orient='index').reset_index().to_dict('records')
    
    


#!!!!!!!!!!!!!https://github.com/Coding-with-Adam/Dash-by-Plotly/blob/master/Dash_Interactive_Graphs/Scatter_mapbox/recycling.py
app = dash.Dash()
#data_points_geojson
#geojson_filter = assign("function(feature, context){return context.props.hideout.includes(feature.properties.trace_name);}")
#default_available_earthquake_forms = df_test.head(1).index.tolist()

cities = [dict(name="Aalborg", lat=57.0268172, lon=9.837735),
          dict(name="Aarhus", lat=56.1780842, lon=10.1119354),
          dict(name="Copenhagen", lat=55.6712474, lon=12.5237848)]
# Create drop down options.
dd_options = [dict(value=c["name"], label=c["name"]) for c in cities]
dd_defaults = [o["value"] for o in dd_options]
cities = [dict(name="Aalborg", lat=57.0268172, lon=9.837735),
          dict(name="Aarhus", lat=56.1780842, lon=10.1119354),
          dict(name="Copenhagen", lat=55.6712474, lon=12.5237848)]
# Create drop down options.
dd_options = [dict(value=c["name"], label=c["name"]) for c in cities]
dd_defaults = [o["value"] for o in dd_options]
print(dd_defaults)
# Generate geojson with a marker for each city and name as tooltip.
geojson = dlx.dicts_to_geojson([{**c, **dict(tooltip=c['name'])} for c in cities])
# Create javascript function that filters on feature name.
geojson_filter = assign("function(feature, context){return context.props.hideout.includes(feature.properties.name);}")
#print(dd_defaults)


image_filename = 'samples_for_wireframe/map_screenshot.PNG'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
#app.layout =html.Div([
app.layout = html.Div([dbc.Row([

                                #Filters|
                                html.Div(style={'width':'20%','marginLeft': 5, 'marginRight': 10,'display':'inline-block',"border":"2px black solid",'vertical-align': 'top'},
                                         children = [
                                            #html.Div([
                                            # dcc.Graph: maybe define somehow here?
                                            #dcc.Graph(figure=fig, id='map-layout')]),
                                             #'width':'15%','marginLeft': 20, 'marginRight': 20,
                                            html.Div(style={ 'display': 'block','vertical-align':'middle'},
                                                     children = [
                                                        html.Div(id="state"), html.Div(id="capital"),
                                                        html.Br(),
                                                        html.H3('Select a date interval'),
                                                        html.Br(),
                                                    #Datepicker to filter for intervals
                                                        dcc.DatePickerRange(
                                                            id='date-filter',
                                                            #min_date_allowed=date(1993, 1, 10),
                                                            min_date_allowed=min_date,
                                                            max_date_allowed=max_date,
                                                            #max_date_allowed=date(2019, 9, 19),
                                                            initial_visible_month=min_date,
                                                            start_date=min_date,
                                                            end_date=max_date
                                                        )]),
                                            #'width':'15%','marginLeft': 20, 'marginRight': 20,'marginTop':30,
                                            html.Div(
                                                style={ 'display': 'block','vertical-align':'middle'},
                                                children = [
                                                html.Br(),
                                                html.H3('Select earthquake depth interval ( in km)'),
                                                html.Br(),
                                                #slider to filter for depth
                                                dcc.RangeSlider(min = min_depth, max = max_depth,
                                                                id='depth-slider',
                                                                marks={i: '{:.2f}'.format(i) for i in depth_space},
                                                                value=[min_depth, max_depth],
                                                                dots=False,
                                                                #step=
                                                                step=0.01,
                                                                updatemode='drag',
                                                                tooltip={"placement": "bottom", "always_visible": False}
                                            )]),
                                             #'width':'15%','marginLeft': 20, 'marginRight': 20,'marginTop':30,
                                            html.Div(
                                                style={'display': 'block','vertical-align':'middle'},
                                                children = [
                                                html.Br(),
                                                html.H3('Select earthquake magnitude interval ( in km)'),
                                                html.Br(),
                                                #slider to filter for magnitudes
                                                dcc.RangeSlider(min = min_magnitude, max = max_magnitude,
                                                                id='magnitude-slider',
                                                                marks={i: '{:.2f}'.format(i) for i in magnitude_space},
                                                                value=[min_magnitude, max_magnitude],
                                                                dots=False,
                                                                step=0.01,
                                                                updatemode='drag',
                                                                tooltip={"placement": "bottom", "always_visible": False}
                                                                ),
                                                ]),
                                             html.Div(
                                                 style={'display': 'block','vertical-align':'middle'},
                                                 children = [
                                                             html.Button('Apply Filter', id='filter-apply-btn', n_clicks=0),
                                                             html.Button('Reset Filter', id='filter-reset-btn', n_clicks=0)
                                                             ]
                                             )
                                         ]),
                                #Map
                                html.Div(
                                        style={'width':'75%', 'marginRight': 5,'display':'inline-block',"border":"2px black solid"},
                                        #children = [dcc.Graph(figure=fig, id='map-layout')])]
                                        #children = [html.Img(src ='data:image/png;base64,{}'.format(encoded_image.decode()))])]
                                        #Todo: consider using bigger outzoom in the beginning, in order to have quicker loading
                                        children = [dl.Map(children = [dl.TileLayer()
                                        ,dl.GeoJSON(data=geojson, options=dict(filter=geojson_filter), hideout=dd_defaults, id="earthquake_events_geojson", zoomToBounds=True)
                                        #dl.GeoJSON(data = data_points_geojson, cluster=True , zoomToBoundsOnClick=True, options = dict(filter=geojson_filter),
                                        #dl.GeoJSON(data = geojson
                                        #,
                                        # cluster=True ,
                                        # zoomToBoundsOnClick=True, options = dict(filter=geojson_filter),
                                        # hideout = dd_defaults,
                                        #    ,superClusterOptions={"radius": 100},
                                        #d ='earthquake_events_geojson')
                                        ], style={'width': '1000px', 'height': '500px'})])]
                                        #, style={'width': '1000px', 'height': '500px'})])]
                                        #children = [ dash_table.DataTable(data = df_test.head(5).to_dict('records'),columns=[{"name": i, "id": i} for i in df_test.columns], id='tbl')])]
                                ),
                dbc.Row(html.Div([
                    #Audio player
                    html.Div(style={'width':'40%','marginLeft': 5,'marginTop': 20, 'marginRight': 20,'display':'inline-block',"border":"2px black solid",'vertical-align': 'top'},
                             #children = [html.Audio(src='/assets/test_raw_waveform_normalized.wav', controls=True)]),
                             #children = [html.Audio(html.Source(src=f"https://file-examples-com.github.io/uploads/2017/11/file_example_WAV_1MG.wav",
                           #type="audio/wav"), controls=True)]),
                           children = [
                                     #html.Div(id = 'audio_player_div', style={'display':'inline-block'}, children =[]),
                                     #!!!! that one is the best below
                                     #html.Audio(src = None, controls=True, id = 'audio_player_main'), 
                                     html.Audio(html.Source(src=f'/assets/test_raw_waveform_normalized_upsampled.wav',type='audio/wav'), controls=True, id = 'audio_player_main'), 
                                     #html.Source(src=f"https://file-examples-com.github.io/uploads/2017/11/file_example_WAV_1MG.wav",
                                        #type="audio/wav")
                           
                                      dash_table.DataTable(id = 'event_info_table',style_header={'display':'none'}
                                      ,style_cell={"whiteSpace": "pre-line"}
                                      
                                      ,data = create_event_infos('HPC.NC_20130403221010_EV')
                                      ,style_cell_conditional=[
                                        {'if': {'column_id': 'index'},
                                        'width': '20%'},
                                        {'if': {'column_id': '0'},
                                        'width': '80%'},
                                    ]
                                      ,columns=[{"name": i, "id": i} for i in ['index','0']])]),
                           #dash_table.DataTable(data = create_event_infos('HPC.NC_20130403221010_EV'),columns=[{"name": i, "id": i} for i in info_colnames], id='event_info_tbl')]),
#                           ]),
                                    #,dash_table.DataTable(data = create_event_infos('HPC.NC_20130403221010_EV'),columns=[{"name": i, "id": i} for i in info_colnames], id='event_info_tbl')]),

                    #Spectrogram
                    html.Div(style={'width':'50%', 'marginRight': 10 ,'marginTop': 20, 'display':'inline-block',"border":"2px black solid"},
                             #children = [html.Img(src='samples_for_wireframe/img.png')])]))])
                             children = [
                                         html.Div(style={ 'display': 'block','vertical-align':'middle'},id = 'seismogram-div',
                                          children = [html.Img(id ='seismogram_img', src = base_seismogram)]),
                                         #html.Div(style={ 'display': 'block','vertical-align':'middle'},id = 'filtered_seismogram-div', children = [html.Img(src = fig_to_uri(filtered_seismogram.plot()))]),
                                         html.Div(style={ 'display': 'block','vertical-align':'middle'},id = 'spectrogram-div',
                                          children = [html.Img( id ='spectrogram_img',src = spectrogram_plot)])
                                         #html.Div(style={ 'display': 'block','vertical-align':'middle'},id = 'spectrogram-div', children = [html.Img(src = fig_to_uri(filtered_spectrogram))])                                                                                                                        
                                        ])]))
                                        
                #,dbc.Row(html.Div(children = [html.Div(style={ 'display': 'block','vertical-align':'middle'},
                #                              children = [dash_table.DataTable(style_header={'display':'none'},data = create_event_infos('HPC.NC_20130403221010_EV'),columns=[{"name": i, "id": i} for i in ['index','0']])])]))                                    
                                              #children = [dash_table.DataTable(data = create_event_infos('HPC.NC_20130403221010_EV'),columns=[{"name": i, "id": i} for i in ['index','0']])])]))                                    
                                              #children = [dash_table.DataTable(data = df_test.head(1).to_dict('records'),columns=[{"name": i, "id": i} for i in df_test.columns])])]))                                    
                    ])

@app.callback(
    #Output(component_id='tbl', component_property='data'), #Output will be the table (later map)
    Output(component_id='depth-slider', component_property= 'value'),
    Output(component_id='magnitude-slider', component_property= 'value'),
    Output(component_id='date-filter', component_property= 'start_date'),
    Output(component_id='date-filter', component_property= 'end_date'),
    Output(component_id='earthquake_events_geojson', component_property= 'hideout'),
    Input(component_id='filter-apply-btn', component_property='n_clicks'), #Input button triggers the callback
    Input(component_id='filter-reset-btn', component_property='n_clicks'), #Input button triggers the callback
    State(component_id='depth-slider', component_property= 'value'),
    State(component_id='magnitude-slider', component_property= 'value'),
    State(component_id='date-filter', component_property= 'start_date'),
    State(component_id='date-filter', component_property= 'end_date')

)
def apply_filter(apply_click,reset_click, depth_value, magnitude_value,start_date,end_date):


    # https://towardsdatascience.com/multi-faceted-data-exploration-in-the-browser-using-leaflet-and-amcharts-f74d049d78d9
    ctx = dash.callback_context
    clicked_element = ctx.triggered[0]['prop_id'].split('.')[0]
    if  clicked_element == 'filter-apply-btn':
        filter_records = df_test['time'].between(pd.to_datetime(start_date),pd.to_datetime(end_date)) & df_test['source_magnitude'].between(magnitude_value[0],magnitude_value[1])\
                         & df_test['source_depth_km'].between(depth_value[0],depth_value[1])
        df_filtered = df_test[filter_records]

        filtered_wave_names =  df_filtered.index.tolist()

        #'earthquake_events_geojson'
        return [depth_value,magnitude_value,start_date,end_date,filtered_wave_names]
        #return [df_filtered.head(5).to_dict('records'),depth_value,magnitude_value,start_date,end_date]

    elif clicked_element == 'filter-reset-btn':
        return [[min_depth,max_depth],[min_magnitude,max_magnitude],min_date,max_date,default_available_earthquake_forms]
       # return [df_test.head(5).to_dict('records'),[min_depth,max_depth],[min_magnitude,max_magnitude],min_date,max_date]

    else:
        return dash.no_update


@app.callback(Output("event_info_table", "data"),
Output('seismogram_img', 'src'),
Output('spectrogram_img', 'src'), 
Output('audio_player_main', 'src'),
Output('audio_player_main', 'style'),
Input("earthquake_events_geojson", "click_feature"),
State('seismogram_img', 'src'),
State('spectrogram_img', 'src'),
State('audio_player_main', 'style')
)

def select_event(clicked_event, current_seismogram_image, current_spectrogram_image, audi_div_element):
    if clicked_event is not None and clicked_event['properties']['cluster'] is False:      
        #Find the corresponding record from metadata
        selected_trace_name = clicked_event['properties']['trace_name']
        table_data = create_event_infos(selected_trace_name)
        selected_event = df_test.loc[selected_trace_name]

        
        provider_of_waveform = stations_df[(stations_df['network_name'] == selected_event['network_code']) &  (stations_df['station_name'] == selected_event['receiver_code'])]
        try:
            client = Client(provider_of_waveform['provider'].tolist()[0])

            wave = extract_waveform(client, selected_event)

            spectrogram_plot = spectrogram_to_uri(wave)
            
            #wave.filter("highpass", freq=0.5).spectrogram(log = True)

            #seismic_plot = fig_to_uri(base_seismogram.plot())
            seismic_plot = fig_to_uri(wave.plot())

            print('seismic sound processing..')
            #create_seismic_sound_to_dash(wave)
            #create_seismic_sound_to_dash_bytes(wave)
            print('FINISHED')
            audio_src = html.Source(src=create_seismic_sound_to_dash_bytes(wave),type='audio/wav')
            audio_src = create_seismic_sound_to_dash_bytes(wave)
            #audio_player_div_player = html.Audio(children = audio_src, controls=True, id ='seismic_audio_player')
            audio_player_style = {}

            #audio_src = html.Source(src=f'/assets/extracted_seismic_sound.wav',type='audio/wav')
            

        except:
            return dash.no_update
            #src = 
                                         #html.Div(style={ 'display': 'block','vertical-align':'middle'},id = 'filtered_seismogram-div', children = [html.Img(src = fig_to_uri(filtered_seismogram.plot()))]),
        #spectrogram_plot =  spectrogram_to_uri(st[2])
        

        return [table_data,seismic_plot,spectrogram_plot,audio_src,audio_player_style]

    else:
        return dash.no_update

        #Load the corresponding informations

        
         
        #some_message = str()
        #for k,v in feature['properties'].items():
        #    some_message += f'Key: {str(k)}'
        #    some_message += '  '
        #    some_message += f'Value: {str(v)}'

        #return f'{some_message}'
        #return f'{str(feature)}'
        #return f"You clicked {feature['properties']['trace_name']}"
        #return f"{df_test.loc[feature['properties']['trace_name']]}"

'''
selection of earthquake events on maps
First, selection in the table will be used as test for composition,
then refactor for map event selection
'''

#app.run_server(debug=True, use_reloader=False,dev_tools_hot_reload=False)  # Turn off reloader if inside Jupyter
app.run_server(debug=True, use_reloader=False)


<ipython-input-1-3ce39731f02a>:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-1-3ce39731f02a>:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-3ce39731f02a>:10: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


['Aalborg', 'Aarhus', 'Copenhagen']


NameError: name 'dlx' is not defined